trainX,trainy,valX,valy 꼴이 아니라서 작업 손대기 어려움.\
파일이 아닌 여러개 나뉜 디렉토리라서 작업 손대기 어려움.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random

from PIL import Image
import tensorflow as tf
# import tensorflow_datasets as tfds
from keras import utils,layers,preprocessing, Sequential
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
import cv2                  

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'cv2'

In [ ]:
print(tf.__version__)

## 공통 함수

In [ ]:
def myPlot(fit_res) :
    key_ = list(fit_res.history.keys())
    plt.figure(figsize=(12,5))
    for i in range(len(key_)):
        plt.plot(fit_res.history[key_[i]], label=key_[i])
    plt.title("metrics")
    plt.xlabel("epoch")
    plt.ylabel("score")
    plt.legend()
    plt.show()

In [ ]:
def myscore(X,y,fit_res=None):
    # test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)
    # print(f"loss: {test_loss:.5f} accuracy: {test_acc:.5f}" )
    history = model.evaluate(X,  y, verbose=1)
    print(f"loss: {history[0]:.5f} accuracy: {history[1]:.5f}" )
    if fit_res != None:
        myPlot(fit_res)
# myscore(test_images,  test_labels, fit_res)

In [ ]:
def myfit(X, y, SPLIT__=0.1, BATCH_SIZE__=128, EPOCHS__=10, ):
    fit_res = model.fit(X, y, batch_size=BATCH_SIZE__, epochs=EPOCHS__, validation_split=SPLIT__)
    return fit_res

In [ ]:
def mypredit_check(test_X, idx=0):
    pred = model.predict(test_X)
    print(pred[idx])
    pred_label = np.argmax(pred[idx])
    print(pred_label, LABEL_NAME__[pred_label])
    plt.imshow(test_X[idx])
    plt.show()

In [ ]:
# np.random.seed(121)
# tf.random.set_seed(121)

# 데이터 로드 및 탐색 [방법1]

### DATASETS LOAD

  '클래스 당 5개의 하위 디렉토리로 구성되어 있다'

flowers_photos/\
      daisy/\
    dandelion/\
    roses/\
  sunflowers/\
  tulips/

# [데이터 로드 및 탐색]

## - 데이터 로드

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = utils.get_file(origin=dataset_url,
                          fname= 'flower_photos',
                          untar=True)
data_dir = pathlib.Path(data_dir)
print(data_dir, type(data_dir))

In [ ]:
def assign_label(img,flower_type):
    return flower_type

In [ ]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        IMG.append(np.array(img))
        LABEL.append(str(label))

In [ ]:
IMG=[]
LABEL=[]
IMG_SIZE=120
FLOWER_DAISY_DIR= str(data_dir)+'/daisy'
FLOWER_SUNFLOWER_DIR= str(data_dir)+ '/sunflowers'
FLOWER_TULIP_DIR= str(data_dir)+ '/tulips'
FLOWER_DANDI_DIR= str(data_dir)+ '/dandelion'
FLOWER_ROSE_DIR= str(data_dir)+ '/roses'


In [ ]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(IMG))

In [ ]:
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(IMG))

In [ ]:
make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(IMG))

In [ ]:
make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(IMG))

In [ ]:
make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(IMG))

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(IMG,LABEL,test_size=0.2,random_state=121)

## - 데이터 탐색

In [ ]:
len(train_images[0]) # 이미지 사이즈 150으로 설정함
# IMG[0]

In [ ]:
train_images[0]
train_labels[:5]

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(train_images[i])
  plt.title(train_labels[i])
  plt.axis("off")

In [ ]:
# import random
# fig,ax=plt.subplots(2,5)
# fig.set_size_inches(10,10)
# for i in range(2):
#     for j in range (5):
#         l=random.randint(0,len(LABEL))
#         ax[i,j].imshow(IMG[l])
#         ax[i,j].set_title(LABEL[l])
        
# plt.tight_layout()

## EDA (keras)

In [ ]:
# print(os.listdir('/root/.keras/datasets/flower_photos')[1:]) # 0에는 txt 포함되어 있음

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))   # photos/*/*.jpg 꼴. 5개 클라스(전부)/파일(전부).jpg 가져와
print(image_count)


In [ ]:
roses = list(data_dir.glob('roses/*'))

In [ ]:
roses[:5]

In [ ]:
Image.open(roses[0])


# 데이터 전처리

In [ ]:
le=LabelEncoder()
LABEL=le.fit_transform(LABEL)
LABEL=to_categorical(LABEL,5)
IMG=np.array(IMG)
IMG=IMG/255

In [ ]:
LABEL[:10]

In [ ]:
# 다시 나누기 (위에는 셔플 후 train 데이터만 EDA 하기 위한 split, 지금은 학습을 위한 split)
train_X, test_X, train_y, test_y = train_test_split(IMG,LABEL,test_size=0.2,random_state=121)

In [ ]:
# train_y[:5] # [2,2,2,0,1] 

In [ ]:
train_y[:10] # [2,2,2,0,1]

# seed 설정

In [ ]:
np.random.seed(121)
random.seed(121)
tf.random.set_seed(121)

## 매개변수 정의

In [ ]:
BATCH_SIZE__= 150
EPOCHS__=10
SPLIT__=0.2
CLASS__=5
LABEL_NAME__ = ['DAISY', 'ROSE', 'Dandelion', 'Sunflower', 'Tulip']

# 데이터 학습

In [ ]:
def myPlot(res) :
    key_ = list(res.history.keys())
    plt.figure(figsize=(12,5))
    for i in range(len(key_)):
        plt.plot(res.history[key_[i]], label=key_[i])
    plt.title("metrics")
    plt.xlabel("epoch")
    plt.ylabel("score")
    plt.legend()
    plt.show()

In [ ]:
def myscore(X,y,fit_res=None):
    # test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)
    # print(f"loss: {test_loss:.5f} accuracy: {test_acc:.5f}" )
    history = model.evaluate(X,  y, verbose=1)
    print(f"loss: {history[0]:.5f} accuracy: {history[1]:.5f}" )
    if fit_res != None:
        myPlot(fit_res)
# myscore(test_images,  test_labels, fit_res)

In [ ]:
def myfit(X, y, SPLIT__=0.1, BATCH_SIZE__=128, EPOCHS__=10, ):
    res = model.fit(X, y, batch_size=BATCH_SIZE__, epochs=EPOCHS__, validation_split=SPLIT__)
    return res

In [ ]:
def mypredit_check(test_X, idx=0):
    pred = model.predict(test_X)
    print(pred[idx])
    pred_label = np.argmax(pred[idx])
    print(pred_label, LABEL_NAME__[pred_label])
    plt.imshow(test_X[idx])
    plt.show()

In [ ]:
print(train_X.shape,test_X.shape,train_y.shape,test_y.shape)

# train 2936 , test 734 로 분할됨 (사진 개수)
# 180*180 사이즈 , RGB 3 
# 5개 클래스 
INPUT_SIZE = (IMG_SIZE,IMG_SIZE,3)

In [ ]:
model = Sequential([
        # CNN
        Conv2D(128, kernel_size=(3, 3),input_shape = INPUT_SIZE, activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
#         Conv2D(32, kernel_size=(3, 3),activation='relu'),
#         MaxPooling2D(pool_size=(2,2)),
        Conv2D(64, kernel_size=(3, 3),activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(32, kernel_size=(3, 3),activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(CLASS__, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


In [ ]:
# 모델 저장 폴더
import os
if not os.path.exists("./model"):
    os.mkdir('./model')

filepath = "./model/{epoch:02d}{val_loss:.3f}.h5"
checkpnt = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
estop = EarlyStopping(monitor='val_loss', patience=4)


In [1]:
res = model.fit(train_X,train_y, batch_size= BATCH_SIZE__, validation_data = (test_X,test_y), epochs=EPOCHS__, callbacks = [checkpnt,estop])

NameError: name 'model' is not defined

In [ ]:
# score 확인
key_ = list(res.history.keys())
plt.figure(figsize=(12,5))
for i in range(len(key_)):
    plt.plot(res.history[key_[i]], label=key_[i])
plt.title("metrics")
plt.xlabel("epoch")
plt.ylabel("score")
plt.legend()
plt.show()

## 예측모델 서비스

In [ ]:
# {'DAISY':0 , 'ROSE':!, 'Dandelion':2, 'Sunflower':3, 'Tulip':4}

In [ ]:
# def mypredit_check(test_X, idxs):
#     pred = model.predict(test_X)
#     print(pred[idx])
#     pred_label = np.argmax(pred[idx])
#     print(pred_label, LABEL_NAME__[pred_label])
#     plt.imshow(test_X[idx])
#     plt.show()


def mypredit_mcheck(prm_X, idxs):
    pred = model.predict(prm_X)
    for i in idxs:
        print(pred[i])
        pred_label = np.argmax(pred[i])
        print(pred_label, LABEL_NAME__[pred_label])
        plt.imshow(prm_X[i])
        plt.show()

In [ ]:
# img = (np.expand_dims(test_X[3],0))
# print(img.shape)
idxs = [3,7,8,12,15,33,36,38,42,44,45]
mypredit_mcheck(test_X, idxs)